In [1]:
from langchain.document_loaders import JSONLoader

In [4]:
loader = JSONLoader(
    file_path='../data/32_residential_zones.jsonl',
    jq_schema='.',
    text_content=False,
    json_lines=True)

In [5]:
# Split documents

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
data = loader.load()
splits = text_splitter.split_documents(data)

In [6]:
# Embed and store splits

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [7]:
# Prompt 

from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

ChatPromptTemplate(input_variables=['question', 'context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [8]:
# LLM

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)

In [9]:
# RAG chain 

from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

## Scenario 1: Permit not needed

In [10]:
rag_chain.invoke("Do I need a permit to build a bed and breakfast? And what are the conditions")

AIMessage(content='No, you do not need a permit to build a bed and breakfast. However, there are conditions that must be met: no more than 10 persons may be accommodated away from their normal place of residence and at least 1 car parking space must be provided for each 2 persons.')

In [16]:
rag_chain.invoke("Do I need a permit to build a B&B? And what are the conditions")

AIMessage(content='No, you do not need a permit to build a Bed and Breakfast (B&B). However, there are conditions that must be met: no more than 10 persons may be accommodated away from their normal place of residence and at least 1 car parking space must be provided for each 2 persons.')

In [11]:
rag_chain.invoke("Do I need a permit to build a Automated collection point?")

AIMessage(content='No, a permit is not required to build an Automated collection point. However, it must meet the requirements of Clause 52.13-3 and 52.13-5, and the gross floor area of all buildings must not exceed 50 square metres.')

## Scenario 2: Out of scope; not in regulations

In [13]:
rag_chain.invoke("Do I need a permit to build a Funland Symposium?")

AIMessage(content='The documents do not provide specific information on whether a permit is required to build a Funland Symposium.')

## Scenario 3: Permit needed

In [17]:
rag_chain.invoke("Do I need a permit to build a Car park?")

AIMessage(content='Yes, you do need a permit to build a car park. The permit is required under Section 2 of the regulations. The car park must be used in conjunction with another use in Section 1 or 2.')